<a href="https://colab.research.google.com/github/JavZon/Machine_Learning/blob/main/ColumnTransformer_str_to_num.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

Datasetni chaqirib olamiz

In [34]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Datasetni 2 qismga bo'lamiz (train va test setlarga)

In [35]:
from sklearn.model_selection import train_test_split
train_set, test_set=train_test_split(df,test_size=0.2, random_state=42)

In [36]:
housing = train_set.drop(columns='median_house_value')
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop(columns='ocean_proximity')

In [37]:
print(housing.head(1))
housing_num.head(1)

       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
14196    -117.03     32.71                33.0       3126.0           627.0   

       population  households  median_income ocean_proximity  
14196      2300.0       623.0         3.2596      NEAR OCEAN  


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596


In [38]:
housing_num.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563


### **Transformer** yasaymiz. Bu transformer har bir uydagi xonalar soni (rooms_per_household) va har bir uydagi odamlar sonini( population_per_household) aniqlab beradi

In [39]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      pass
    def fit(self, X, y=None):
      return self
    def transform(self, X):
      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
      population_per_household=X[:, population_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household]

Endi esa NaN larning o'rnini to'ldirish uchun **SimpleImputer** va datasetni -1 va 1 oralig'iga keltirish uchun **StandardScaler**ni chaqiramiz

In [40]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('custom_obj_adder', CustomTransformer()),
    ('std_scaler', StandardScaler())
])

numeric_pipeline.fit_transform(housing_num)


array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.326196  ,
        -0.17491646,  0.05137609],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.03584338,
        -0.40283542, -0.11736222],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.14470145,
         0.08821601, -0.03227969],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.49697313,
        -0.60675918,  0.02030568],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.96545045,
         0.40217517,  0.00707608],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.68544764,
        -0.85144571, -0.08535429]])

**array** bo'yicha takrorlash

In [41]:
arr=np.random.randint(1,100, size=(4,5))

In [42]:
arr
#      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
#      population_per_household=X[:, population_ix] / X[:, households_ix]

array([[42, 38, 71, 68, 89],
       [ 8, 78, 11, 72, 56],
       [ 2, 48,  6, 10, 89],
       [53, 90, 21, 43, 17]])

In [43]:
arr[:,3]

array([68, 72, 10, 43])

In [44]:
arr[2,:]

array([ 2, 48,  6, 10, 89])

**Columnlarni** tekshirib olamiz

In [45]:
housing_num.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income'],
      dtype='object')

In [46]:
cols=list(housing_num.columns)

In [47]:
cols

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

**ColumnTransformer** nomli yangi classni chaqirib olamiz sklearn.compose ning ichidan.

In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols=list(housing_num.columns)
string_cols=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('numeric', numeric_pipeline, num_cols),
    ('string_cat',OneHotEncoder(), string_cols)
])

In [50]:
housing_full_dataset=full_pipeline.fit_transform(housing)

In [51]:
housing_full_dataset

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

In [52]:
housing_full_dataset[:3,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])